In [180]:
class Tree():
    def __init__(self, root=None):
        self.root = root
        self.size = 0
        
    def insert(self, node, start_node=None):
        # Create root if no root
        if start_node == None:
            start_node = self.root
        if not self.root:
            self.root = node
        else: 
            if node.key < start_node.key: # recurse left
                if start_node.has_left_child(): # recurse
                    self.insert(node, start_node.left_child)
                else:
                    start_node.left_child = node
                    node.parent = start_node
            else: 
                if start_node.has_right_child():
                    self.insert(node, start_node.right_child)
                else:
                    start_node.right_child = node
                    node.parent = start_node
        self.size += 1
        
    def search(self, key, start_node=None):
        if start_node == None:
            start_node = self.root
        if key == start_node.key:
            return start_node
        # If we have reached the end but our node has no children, return
        if not (start_node.has_left_child() or start_node.has_left_child()):
            return None
        if key < start_node.key:
            return self.search(key, start_node.left_child)
        else:
            return self.search(key, start_node.right_child)
        
    def traverse_in_order(self, start_node = None):
        if start_node == None:
            start_node = self.root
        if start_node.has_left_child():
            self.traverse_in_order(start_node.left_child)
        print(start_node.key)
        if start_node.has_right_child():
            self.traverse_in_order(start_node.right_child)        
    
    def predecessor(self, node_key):
        start_node = self.search(node_key)
        if start_node.key == self.minimum:
            return start_node
        
        # Case 1: we have a left child
        if start_node.has_left_child():
            new_node = start_node.left_child
            while new_node.has_right_child():
                new_node = new_node.right_child
            return new_node
                
        # Case 2: we do not have a left child
        else:
            parent_node = start_node.parent
            while parent_node.key > start_node.key:
                parent_node = parent_node.parent
            return parent_node

        
    def successor(self, node_key):
        start_node = self.search(node_key)
        if start_node.key == self.maximum:
            return start_node
        
        # Case 1: we have a left child
        if start_node.has_right_child():
            new_node = start_node.right_child
            while new_node.has_left_child():
                new_node = new_node.left_child
            return new_node
                
        # Case 2: we do not have a left child
        else:
            parent_node = start_node.parent
            while parent_node.key < start_node.key:
                parent_node = parent_node.parent
            return parent_node
        

    def minimum(self):
        node = self.root
        while node.has_left_child():
            node = node.left_child
        return node.key

    
    def maximum(self):
        node = self.root
        while node.has_right_child():
            node = node.right_child
        return node.key

    
    def delete(self, node_key):
        node = self.search(node_key)
        if not node:
            return None

        # Case 1: no children
        if not node.has_left_child() and not node.has_right_child():
            if node == node.parent.left_child:
                node.parent.left_child = None
            else:
                node.parent.right_child = None
            del node

        elif node.has_left_child() and not node.has_right_child():
            # Node's left child takes position of node
            if node.parent:
                if node == node.parent.left_child:
                    node.parent.left_child = node.left_child
                else:
                    node.parent.right_child = node.right_child
                node.left_child.parent = node.parent
            del node
        elif not node.has_left_child() and node.has_right_child():
            # Node's right child takes position of node
            if node.parent:
                if node == node.parent.left_child:
                    node.parent.left_child = node.right_child
                else:
                    node.parent.right_child = node.right_child
                node.right_child.parent = node.parent
            del node
        # Case with two keys
        else:
            predecessor_node = self.predecessor(node_key)
            
            # Need to handle case where predecessor is node's immediate child
            # slightly differently
            if predecessor_node != node.left_child:
                predecessor_node.left_child = node.left_child 
            if predecessor_node != node.right_child:
                predecessor_node.right_child = node.right_child

            # Get rid of predecessor node's parent's child
            if predecessor_node.parent:
                if predecessor_node == predecessor_node.parent.left_child:
                    predecessor_node.parent.left_child = None
                else:
                    predecessor_node.parent.right_child = None  

            # Predecessor node gets new parent, even if "None"
            predecessor_node.parent = node.parent

            # Node's old parent gets predecessor node as a child
            if node.parent:
                if node == node.parent.left_child:
                    node.parent.left_child = predecessor_node
                else:
                    node.parent.right_child = predecessor_node
            # If no parent, predecessor node is the root
            else:
                self.root = predecessor_node
            
    
    def traverse_in_order(self, start_node = None):
        if start_node == None:
            start_node = self.root
        if start_node.has_left_child():
            self.traverse_in_order(start_node.left_child)
        print(start_node.key)
        if start_node.has_right_child():
            self.traverse_in_order(start_node.right_child)        
                        

class Node():

    def __init__(self, key, payload, parent=None, left_child=None, right_child=None):
        '''
        Keys used for sorting, value used for returning.
        '''
        self.key = key
        self.payload = payload
        self.parent = parent
        self.left_child = left_child
        self.right_child = right_child   
        
    def has_left_child(self):
        return True if self.left_child else False
        
    def has_right_child(self):
        return True if self.right_child else False

In [181]:
t = Tree()
t.insert(Node(5, 'a'))
t.insert(Node(3, 'b'))
t.insert(Node(4, 'c'))
t.insert(Node(2, 'd'))
t.insert(Node(7, 'd'))
t.insert(Node(6, 'd'))
t.insert(Node(8, 'd'))
t.insert(Node(9, 'd'))
print("Traversing in order:")
t.traverse_in_order()
print("Testing:")
print(t.search(3).key) # 3
print(t.predecessor(3).key) # 2
# t.delete(3)
# t.traverse_in_order()

Traversing in order:
2
3
4
5
6
7
8
9
Testing:
3
2


## Testing deleting

### No children

In [182]:
node = t.search(2)
print(node.has_left_child())
print(node.has_right_child())

False
False


In [183]:
t.delete(2)

In [184]:
t.traverse_in_order()

3
4
5
6
7
8
9


### One child

In [185]:
node = t.search(8)
print(node.has_left_child())
print(node.has_right_child())

False
True


In [186]:
t.delete(8)

In [187]:
t.traverse_in_order()

3
4
5
6
7
9


### Two children

In [188]:
node = t.search(5)
print(node.key)
print(node.right_child.key)
print(node.left_child.key)
# print(node.left_child == node)

5
7
3


In [189]:
print("First traversal")
t.traverse_in_order()
t.delete(5)
print("Second traversal")
t.traverse_in_order()

First traversal
3
4
5
6
7
9
Second traversal
3
4
6
7
9


In [190]:
t.traverse_in_order()

3
4
6
7
9


In [191]:
r = t.root
print(r.key)
print(r.left_child.key)
print(r.right_child.key)

4
3
7


Works as expected on simple test cases!